In [17]:
from datetime import datetime

In [1]:
log_path = "../src/db/app.log"

In [2]:
logs = open(log_path, "r").read()

In [11]:
log_lines = logs.split("\n2025")

In [12]:
for index in range(1, len(log_lines)):
    log_lines[index] = {"log": "2025" + log_lines[index]}

In [15]:
log_lines[0] = {"log": log_lines[0]}

In [26]:
for line in log_lines:
    line["datetime"] = datetime.strptime(line["log"].split(",")[0], "%Y-%m-%d %H:%M:%S")

In [28]:
relevant_log_lines = [
    line
    for line in log_lines
    if line["datetime"] >= datetime(2025, 1, 28, 9, 30, 0)
    and line["datetime"] <= datetime(2025, 1, 28, 11, 0, 0)
]

In [33]:
read_queries = [line for line in relevant_log_lines if "SELECT" in line["log"]]

In [34]:
len(read_queries)

10175

In [48]:
write_queries = [line for line in relevant_log_lines if "INSERT" in line["log"]]

In [50]:
other_queries = [
    line
    for line in relevant_log_lines
    if "SELECT" not in line["log"] and "INSERT" not in line["log"]
    # and "UPDATE" not in line["log"]
    # and "DELETE" not in line["log"]
    and "BEGIN" not in line["log"]
    and "COMMIT" not in line["log"]
    and "You are" not in line["log"]
    and "Task:\n" not in line["log"]
]

In [51]:
len(other_queries)

0

In [57]:
time_diffs = [
    relevant_log_lines[index]["datetime"] - relevant_log_lines[index - 1]["datetime"]
    for index in range(1, len(relevant_log_lines))
]

In [59]:
import sqlite3

In [68]:
print(read_queries[0]["log"].split("Executing operation:")[-1].strip())

SELECT MAX(datetime(timestamp, '+5 hours', '+30 minutes')) as timestamp
    FROM chat_history
    WHERE user_id = 3 AND task_id IN (SELECT task_id FROM course_tasks WHERE course_id IN (SELECT course_id FROM course_cohorts WHERE cohort_id = 1))
    GROUP BY DATE(datetime(timestamp, '+5 hours', '+30 minutes'))
    ORDER BY timestamp DESC


In [69]:
def extract_query(log_line):
    """Extract the SQL query from a log line"""
    # Assuming the query is after a comma in the log
    return log_line["log"].split("Executing operation:")[-1].strip()

In [73]:
def measure_query_times(queries, db_path="../src/db/db.sqlite.prod"):
    """Run queries and measure their execution times"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    results = []
    for query_line in queries:
        query = extract_query(query_line)

        # Start timing
        cursor.execute("SELECT sqlite_version()")  # Warm up connection
        start_time = datetime.now()

        # Execute query
        cursor.execute(query)

        # End timing
        end_time = datetime.now()
        execution_time = (end_time - start_time).total_seconds()

        results.append(
            {
                "query": query,
                "execution_time": execution_time,
                "timestamp": query_line["datetime"],
            }
        )

    conn.close()
    return results

In [76]:
read_query_times = measure_query_times(read_queries)

In [78]:
times = [time["execution_time"] for time in read_query_times]

In [79]:
import numpy as np

# Calculate percentiles
p90 = np.percentile(times, 90)
p95 = np.percentile(times, 95)
p99 = np.percentile(times, 99)

print(f"P90: {p90:.3f}s")
print(f"P95: {p95:.3f}s")
print(f"P99: {p99:.3f}s")

P90: 0.001s
P95: 0.001s
P99: 0.015s


In [80]:
write_query_times = measure_query_times(write_queries)

In [82]:
times = [time["execution_time"] for time in write_query_times]

In [83]:
import numpy as np

# Calculate percentiles
p90 = np.percentile(times, 90)
p95 = np.percentile(times, 95)
p99 = np.percentile(times, 99)

print(f"P90: {p90:.3f}s")
print(f"P95: {p95:.3f}s")
print(f"P99: {p99:.3f}s")

P90: 0.000s
P95: 0.000s
P99: 0.000s
